# Covid19 Analysis for Recovery Phase Prediction

Source Datasets:

1. Our World in Data COVID-19 Testing dataset https://ourworldindata.org/coronavirus-testing
1. Google Covid19 Community Mobility Reports https://www.google.com/covid19/mobility/

Methdology:

1. Aggregate source data at daily country level and consolidate as master data
1. Calculate risk index based on rate of infection and total number of infections
1. Train machine learning model to translate relationship between risk drivers (mobility and/or restriction) and risk index

Prediction Scenario:
1. To prevent second spike in recovery phase, one would be able to predict on the risk index given the assumed risk drivers therefore to adjust government policies and country wide communications

In [3]:
#load coviddata dataset
import pandas as pd
df = spark.table("owid_covid_data_csv")
coviddata = df.select("*").toPandas()
coviddata = coviddata[coviddata['date']>='2020-03-01']

#change data types and column names
coviddata['date'] = pd.to_datetime(coviddata['date'])
coviddata.rename(columns={'iso_code':'iso3_code'}, inplace=True)

display(coviddata)

iso3_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
ABW,Aruba,2020-03-13T00:00:00.000+0000,2,2,0,0,18.733,18.733,0.0,0.0,null,null,null,null,null,null,null,0.0,106766.0,584.8,41.2,13.085,7.452,35973.781,null,null,11.62,null,null,null,null
ABW,Aruba,2020-03-20T00:00:00.000+0000,4,2,0,0,37.465,18.733,0.0,0.0,null,null,null,null,null,null,null,30.56,106766.0,584.8,41.2,13.085,7.452,35973.781,null,null,11.62,null,null,null,null
ABW,Aruba,2020-03-24T00:00:00.000+0000,12,8,0,0,112.395,74.93,0.0,0.0,null,null,null,null,null,null,null,41.67,106766.0,584.8,41.2,13.085,7.452,35973.781,null,null,11.62,null,null,null,null
ABW,Aruba,2020-03-25T00:00:00.000+0000,17,5,0,0,159.227,46.831,0.0,0.0,null,null,null,null,null,null,null,41.67,106766.0,584.8,41.2,13.085,7.452,35973.781,null,null,11.62,null,null,null,null
ABW,Aruba,2020-03-26T00:00:00.000+0000,19,2,0,0,177.959,18.733,0.0,0.0,null,null,null,null,null,null,null,41.67,106766.0,584.8,41.2,13.085,7.452,35973.781,null,null,11.62,null,null,null,null
ABW,Aruba,2020-03-27T00:00:00.000+0000,28,9,0,0,262.256,84.296,0.0,0.0,null,null,null,null,null,null,null,41.67,106766.0,584.8,41.2,13.085,7.452,35973.781,null,null,11.62,null,null,null,null
ABW,Aruba,2020-03-28T00:00:00.000+0000,28,0,0,0,262.256,0.0,0.0,0.0,null,null,null,null,null,null,null,41.67,106766.0,584.8,41.2,13.085,7.452,35973.781,null,null,11.62,null,null,null,null
ABW,Aruba,2020-03-29T00:00:00.000+0000,28,0,0,0,262.256,0.0,0.0,0.0,null,null,null,null,null,null,null,82.41,106766.0,584.8,41.2,13.085,7.452,35973.781,null,null,11.62,null,null,null,null
ABW,Aruba,2020-03-30T00:00:00.000+0000,50,22,0,0,468.314,206.058,0.0,0.0,null,null,null,null,null,null,null,82.41,106766.0,584.8,41.2,13.085,7.452,35973.781,null,null,11.62,null,null,null,null
ABW,Aruba,2020-04-01T00:00:00.000+0000,55,5,0,0,515.145,46.831,0.0,0.0,null,null,null,null,null,null,null,82.41,106766.0,584.8,41.2,13.085,7.452,35973.781,null,null,11.62,null,null,null,null


In [4]:
import country_converter as coco

iso2_code = coco.convert(names=coviddata['iso3_code'].to_list(), to='ISO2', not_found=None)

coviddata['iso2_code'] = iso2_code
coviddata = coviddata[coviddata['iso2_code']!='None']

WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:OWID_WRL not found in regex
WARNING:root:None not found in regex
WARNING:root:None not found in regex
WARNING:root:None not found in regex

In [5]:
#load mobility dataset
df = spark.table("global_mobility_report_csv")
mobility = df.select("*").toPandas()
mobility = mobility[mobility['date']>='2020-03-01']
display(mobility)

country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
AE,United Arab Emirates,null,null,2020-03-01T00:00:00.000+0000,3.0,10.0,2.0,-1.0,4.0,1.0
AE,United Arab Emirates,null,null,2020-03-02T00:00:00.000+0000,0.0,7.0,1.0,-2.0,4.0,1.0
AE,United Arab Emirates,null,null,2020-03-03T00:00:00.000+0000,0.0,6.0,0.0,-5.0,4.0,1.0
AE,United Arab Emirates,null,null,2020-03-04T00:00:00.000+0000,-1.0,7.0,-2.0,-5.0,3.0,2.0
AE,United Arab Emirates,null,null,2020-03-05T00:00:00.000+0000,-3.0,6.0,-2.0,-5.0,3.0,2.0
AE,United Arab Emirates,null,null,2020-03-06T00:00:00.000+0000,-7.0,5.0,-8.0,-9.0,0.0,3.0
AE,United Arab Emirates,null,null,2020-03-07T00:00:00.000+0000,-3.0,6.0,1.0,-8.0,4.0,2.0
AE,United Arab Emirates,null,null,2020-03-08T00:00:00.000+0000,1.0,8.0,6.0,-9.0,-1.0,3.0
AE,United Arab Emirates,null,null,2020-03-09T00:00:00.000+0000,-3.0,4.0,4.0,-10.0,-1.0,4.0
AE,United Arab Emirates,null,null,2020-03-10T00:00:00.000+0000,-4.0,6.0,3.0,-11.0,-2.0,4.0


In [6]:
#change data type
mobility['date']= pd.to_datetime(mobility['date'])

mobility.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390218 entries, 15 to 458564
Data columns (total 11 columns):
country_region_code 390218 non-null object
country_region 390218 non-null object
sub_region_1 378866 non-null object
sub_region_2 225838 non-null object
date 390218 non-null datetime64[ns]
retail_and_recreation_percent_change_from_baseline 297095 non-null float64
grocery_and_pharmacy_percent_change_from_baseline 285598 non-null float64
parks_percent_change_from_baseline 180765 non-null float64
transit_stations_percent_change_from_baseline 208666 non-null float64
workplaces_percent_change_from_baseline 380898 non-null float64
residential_percent_change_from_baseline 210674 non-null float64
dtypes: datetime64[ns](1), float64(6), object(4)
memory usage: 35.7+ MB

In [7]:
#aggregate values by date and country
mobility_agg = mobility.groupby(['date','country_region','country_region_code']).agg(
  {'retail_and_recreation_percent_change_from_baseline':'sum',
   'grocery_and_pharmacy_percent_change_from_baseline':'sum',
   'parks_percent_change_from_baseline':'sum',
   'transit_stations_percent_change_from_baseline':'sum',
   'workplaces_percent_change_from_baseline':'sum',
   'residential_percent_change_from_baseline':'sum'}).reset_index()

mobility_agg.head()

,date,country_region,country_region_code,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,2020-03-01,Afghanistan,AF,3.0,10.0,2.0,10.0,9.0,0.0
1,2020-03-01,Angola,AO,6.0,30.0,-9.0,7.0,6.0,1.0
2,2020-03-01,Antigua and Barbuda,AG,5.0,20.0,22.0,3.0,2.0,0.0
3,2020-03-01,Argentina,AR,-40.0,120.0,-202.0,-102.0,-23.0,64.0
4,2020-03-01,Aruba,AW,8.0,6.0,5.0,3.0,4.0,0.0


In [8]:
#join three dataset by country code

#version1 : use datasets (kaggle covid19, population and mobility)
#before_master = covid19_agg.merge(mobility_agg, left_on=['ObservationDate','CountryCode'], right_on=['date','country_region_code'], how='left')
#master = before_master.merge(population, left_on='country_region_code', right_on='Code', how='left')
#master = master.drop(['ObservationDate', 'Country/Region','CountryCode', 'Index', 'Country', 'Country code', 'Code'], axis=1)
#master = master.dropna(, inplace=True)

#version2： use datasets (owid world coviddata with population included and mobility)
master = coviddata.merge(mobility_agg, left_on=['date','iso2_code'], right_on=['date','country_region_code'], how='left')
master = master.dropna(subset=['country_region_code']) 
master = master.drop(['iso3_code', 'iso2_code'], axis=1)

master

,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k,country_region,country_region_code,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,Aruba,2020-03-13,2,2,0,0,18.733,18.733,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,None,0.00,106766.0,584.800,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,Aruba,AW,-10.0,40.0,-4.0,-5.0,3.0,1.0
1,Aruba,2020-03-20,4,2,0,0,37.465,18.733,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,None,30.56,106766.0,584.800,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,Aruba,AW,-53.0,-7.0,-41.0,-45.0,-41.0,22.0
2,Aruba,2020-03-24,12,8,0,0,112.395,74.930,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,None,41.67,106766.0,584.800,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,Aruba,AW,-55.0,-21.0,-49.0,-57.0,-51.0,26.0
3,Aruba,2020-03-25,17,5,0,0,159.227,46.831,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,None,41.67,106766.0,584.800,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,Aruba,AW,-57.0,-20.0,-56.0,-60.0,-53.0,27.0
4,Aruba,2020-03-26,19,2,0,0,177.959,18.733,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,None,41.67,106766.0,584.800,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,Aruba,AW,-57.0,-15.0,-54.0,-61.0,-55.0,27.0
5,Aruba,2020-03-27,28,9,0,0,262.256,84.296,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,None,41.67,106766.0,584.800,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,Aruba,AW,-50.0,-3.0,-50.0,-55.0,-52.0,25.0
6,Aruba,2020-03-28,28,0,0,0,262.256,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,None,41.67,106766.0,584.800,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,Aruba,AW,-49.0,-3.0,-50.0,-60.0,-51.0,20.0
7,Aruba,2020-03-29,28,0,0,0,262.256,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,None,82.41,106766.0,584.800,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,Aruba,AW,-88.0,-66.0,-80.0,-88.0,-72.0,0.0
8,Aruba,2020-03-30,50,22,0,0,468.314,206.058,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,None,82.41,106766.0,584.800,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,Aruba,AW,-74.0,-43.0,-63.0,-77.0,-71.0,35.0
9,Aruba,2020-04-01,55,5,0,0,515.145,46.831,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,None,82.41,106766.0,584.800,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,Aruba,AW,-70.0,-36.0,-62.0,-71.0,-69.0,34.0


In [9]:
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#plot on residential mobility stats with selected countries in May
plt.figure(figsize=(20,4))
countrys=['GB','US','JP','FR','CA','DE','IN','IT','UA']
master_selection=master[(master['date']>='2020-05-01') & (master['country_region_code'].isin(countrys))]
ax = sns.lineplot(x="date", y="residential_percent_change_from_baseline", hue="country_region_code", data=master_selection)
plt.yscale('log')
display(plt.show())

Risk Index Formula: 

1. rate of infection: the increase/decrease rate of current total cases against total cases from 14 days ago (normal virus probation period)
1. number of infection: current total cases per million

Formula refers to uk government risk index guidance: 
https://www.spectator.co.uk/article/how-number-10-should-illustrate-its-covid-alert-formula

In [11]:
#align data with total cases minus 14 days

#version1: total manual weightage assignment
#master['risk_index'] = (master['Confirmed']/master['Year_2020'])*0.2+(master['Deaths']/master['Year_2020'])*0.6-(master['Recovered']/master['Year_2020'])*0.2

#version2: combined use of spead rate r and number of accumulative infections i
master['country_level_index'] = master.groupby(['country_region_code']).cumcount()+1
 
back_track = []
total_cases_minus14 = []
for country in master['country_region_code'].unique():
  for index in range(0,max(master[master['country_region_code']==country]['country_level_index'])):
      try:
        back_track.append(master.loc[(master['country_region_code']==country) & (master['country_level_index']==(index-14)),'total_cases'].values)
      except KeyError:
        back_track.append(float("nan"))
        
import numpy as np
out = []
for i in range(0,len(back_track)):
  if len(back_track[i])==0:
    out.append(np.array([0]))
  else:
    out.append(back_track[i])
  
from itertools import chain
total_cases_minus14 = list(chain(*out))

master['total_cases_minus14'] = total_cases_minus14

In [12]:
#calcualte the two factors that contribute to risk index
master['rate_of_infection'] = (master['total_cases']-master['total_cases_minus14'])/master['total_cases_minus14']
master['number_of_infection'] = master['total_cases_per_million']

#plot on matrix of rate_of_infection and number_of_infection with a selection of countries
plt.figure(figsize=(6,6))
countrys=['GB','US','JP','FR','CA','DE','IN','IT','UA']
master_selection=master[(master['country_region_code'].isin(countrys))]
ax = sns.scatterplot(x="number_of_infection", y="rate_of_infection", hue="country_region_code", data=master_selection)
display(plt.show())

In [13]:
#calcualte risk index 
import math
risk_index = []

for i in range(0,len(master)):
  if math.isinf(master.iloc[i]['rate_of_infection']) or (master.iloc[i]['rate_of_infection']<20 and master.iloc[i]['number_of_infection']<1000):
    risk_index.append(1)
  elif master.iloc[i]['rate_of_infection']<40 and master.iloc[i]['number_of_infection']<2000:
    risk_index.append(2)
  elif master.iloc[i]['rate_of_infection']<60 and master.iloc[i]['number_of_infection']<3000:
    risk_index.append(3)
  elif master.iloc[i]['rate_of_infection']<80 and master.iloc[i]['number_of_infection']<4000:
    risk_index.append(4)
  else:
    risk_index.append(5)

master['risk_index'] = risk_index

master[master['location']=='United Kingdom']

,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k,country_region,country_region_code,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,country_level_index,total_cases_minus14,rate_of_infection,number_of_infection,risk_index
5405,United Kingdom,2020-03-01,23,5,0,0,0.339,0.074,0.000,0.000,11750.0,1267.0,0.173,0.019,775.0,0.011,people tested,11.11,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,NaN,2.54,United Kingdom,GB,957.0,355.0,700.0,650.0,201.0,-57.0,1,0,inf,0.339,1
5406,United Kingdom,2020-03-02,36,13,0,0,0.530,0.191,0.000,0.000,13525.0,1775.0,0.199,0.026,998.0,0.015,people tested,11.11,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,NaN,2.54,United Kingdom,GB,901.0,679.0,1597.0,454.0,125.0,9.0,2,0,inf,0.530,1
5407,United Kingdom,2020-03-03,40,4,0,0,0.589,0.059,0.000,0.000,13911.0,386.0,0.205,0.006,1017.0,0.015,people tested,11.11,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,NaN,2.54,United Kingdom,GB,790.0,535.0,1429.0,246.0,198.0,-9.0,3,0,inf,0.589,1
5408,United Kingdom,2020-03-04,51,11,0,0,0.751,0.162,0.000,0.000,16659.0,2748.0,0.245,0.040,1361.0,0.020,people tested,11.11,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,NaN,2.54,United Kingdom,GB,497.0,638.0,283.0,46.0,98.0,58.0,4,0,inf,0.751,1
5409,United Kingdom,2020-03-05,85,34,0,0,1.252,0.501,0.000,0.000,18083.0,1424.0,0.266,0.021,1485.0,0.022,people tested,11.11,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,NaN,2.54,United Kingdom,GB,400.0,636.0,737.0,113.0,96.0,72.0,5,0,inf,1.252,1
5410,United Kingdom,2020-03-06,115,30,0,0,1.694,0.442,0.000,0.000,20338.0,2255.0,0.300,0.033,1622.0,0.024,people tested,11.11,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,NaN,2.54,United Kingdom,GB,722.0,841.0,2723.0,595.0,115.0,-11.0,6,0,inf,1.694,1
5411,United Kingdom,2020-03-07,163,48,1,1,2.401,0.707,0.015,0.015,21460.0,1122.0,0.316,0.017,1568.0,0.023,people tested,11.11,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,NaN,2.54,United Kingdom,GB,278.0,419.0,1051.0,409.0,256.0,-5.0,7,0,inf,2.401,1
5412,United Kingdom,2020-03-08,206,43,2,1,3.034,0.633,0.029,0.015,23513.0,2053.0,0.346,0.030,1680.0,0.025,people tested,11.11,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,NaN,2.54,United Kingdom,GB,836.0,498.0,2119.0,911.0,207.0,-64.0,8,0,inf,3.034,1
5413,United Kingdom,2020-03-09,273,67,2,0,4.021,0.987,0.029,0.000,24960.0,1447.0,0.368,0.021,1634.0,0.024,people tested,11.11,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,NaN,2.54,United Kingdom,GB,168.0,572.0,605.0,-66.0,57.0,124.0,9,0,inf,4.021,1
5414,United Kingdom,2020-03-10,321,48,3,1,4.729,0.707,0.044,0.015,26261.0,1301.0,0.387,0.019,1764.0,0.026,people tested,11.11,67886004.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,NaN,2.54,United Kingdom,GB,208.0,824.0,521.0,-206.0,58.0,139.0,10,0,inf,4.729,1


In [14]:
#output dataset master
df = spark.createDataFrame(master)
#df.write.format("csv").saveAsTable("covid_analysis_with_risk_index")
df.write.mode("overwrite").saveAsTable("covid_analysis_with_risk_index")

In [15]:
dbutils.fs.rm("/FileStore/analysis-output/covid_analysis_with_risk_index.csv",True)
df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/analysis-output/covid_analysis_with_risk_index.csv")
#IMPORTANT: file downloadable at:
#https://adb-5999897127967121.1.azuredatabricks.net/files/analysis-output/covid_analysis_with_risk_index.csv/part-00000-tid-5579663262833822991-197e2b9c-ad1e-474d-a172-faa190badc8f-53-1-c000.csv?o=5999897127967121

Prediction Model Build: 

Make use of a classification model to capture relationship between risk drivers (mobility and/or restriction) and risk index to future prediction that aids recovery phase planning

In [17]:
#split data into a labels dataframe and a features dataframe
labels = master['risk_index'].values
featureNames = ['retail_and_recreation_percent_change_from_baseline', 'grocery_and_pharmacy_percent_change_from_baseline',
                'parks_percent_change_from_baseline','transit_stations_percent_change_from_baseline',
                'workplaces_percent_change_from_baseline','residential_percent_change_from_baseline']
features = master[featureNames].values

In [18]:
#normalize features (columns) to have unit variance
from sklearn.preprocessing import normalize
features = normalize(features, axis=0)
features

Out[15]: array([[-2.03827498e-05, 1.94762267e-04, -2.40210972e-05,
 -1.94127371e-05, 3.90722859e-06, 6.45538499e-06],
 [-1.08028574e-04, -3.40833968e-05, -2.46216246e-04,
 -1.74714634e-04, -5.33987908e-05, 1.42018470e-04],
 [-1.12105124e-04, -1.02250190e-04, -2.94258440e-04,
 -2.21305203e-04, -6.64228861e-05, 1.67840010e-04],
 ...,
 [-1.67138549e-04, -1.31464531e-04, -2.70237343e-04,
 -5.08613713e-04, -3.90722859e-05, 2.58215400e-04],
 [-2.26248523e-04, -1.75286041e-04, -3.30290086e-04,
 -6.94975989e-04, -1.30240953e-06, 1.35563085e-04],
 [-3.09817798e-04, -4.82036612e-04, -5.16453589e-04,
 -7.80392032e-04, -8.54380653e-04, 4.77698489e-04]])

In [19]:
#hold out 20% of the data for testing with stratification enabled
from sklearn.model_selection import train_test_split
trainingLabels, testLabels, trainingFeatures, testFeatures = train_test_split(labels, features, test_size=0.2, stratify=labels)
ntrain, ntest = len(trainingLabels), len(testLabels)
print('Split data randomly into 2 sets: %d training and %d test instances.' % (ntrain, ntest))

Split data randomly into 2 sets: 8072 training and 2018 test instances.

In [20]:
#train a gradientboosting model with fixed hyperparameters
from sklearn.ensemble import GradientBoostingClassifier
Clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                 max_depth=2, random_state=0)
Clf.fit(features, labels)
#print('Trained model with fixed random_state = 0')

Out[17]: GradientBoostingClassifier(criterion='friedman_mse', init=None,
 learning_rate=1.0, loss='deviance', max_depth=2,
 max_features=None, max_leaf_nodes=None,
 min_impurity_decrease=0.0, min_impurity_split=None,
 min_samples_leaf=1, min_samples_split=2,
 min_weight_fraction_leaf=0.0, n_estimators=100,
 n_iter_no_change=None, presort='auto', random_state=0,
 subsample=1.0, tol=0.0001, validation_fraction=0.1,
 verbose=0, warm_start=False)

In [21]:
#score the model
TrainingScore, TestScore = Clf.score(trainingFeatures, trainingLabels), Clf.score(testFeatures, testLabels)

print('Model  Training Test')
print('Version 1.0\t%g\t%g' % (TrainingScore, TestScore))

Model Training Test
Version 1.0	0.828048	0.826065

In [22]:
#compare actual vs predicted values
predLabels = Clf.predict(testFeatures)

comparison= pd.DataFrame({'Actual': testLabels.flatten(), 'Predicted': predLabels.flatten()})
comparison

,Actual,Predicted
0,1,1
1,1,1
2,5,2
3,4,1
4,5,3
5,5,1
6,1,1
7,1,1
8,1,1
9,1,1


In [23]:
#plot first 100 comparison results
comparison.head(100).plot(kind='bar',figsize=(20,6))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
display(plt.show())